In [1]:
"""
Since constraints and rules are essentially the same thing, they should be unified into the same type of object. 
Things to consider in this process:

    - The Dwarf Planet constraint is similar to, but not the same as a Band Rule. A Band Rule does not require the
        objects to be at the two ends of the band; they can be anywhere within. It's not an "exact band" but a 
        maximum requirement
    - The Comet constraint is not modeled by any of the rules. A "restricted positions" type rules should be made 
        to accommodate this
    - This will allow the generate_rule/eliminate_sectors methods to take in a set of constraints and compare it to
        themselves to determine if generating a certain rule makes sense
    - It might make sense to unify SelfRule and RelationRule in the process
        - The three types of self rules are equivalent to the three types of relation rules
        - However, the comet-type rule would not be equivalent to any relation rule that I can think of
    - Constraints always had qualifiers of "none" or "every", not "at least one", which affected the fill_board 
        method since it could make fewer assumptions.
"""

'\nSince constraints and rules are essentially the same thing, they should be unified into the same type of object. \nThings to consider in this process:\n\n    - The Dwarf Planet constraint is similar to, but not the same as a Band Rule. A Band Rule does not require the\n        objects to be at the two ends of the band; they can be anywhere within. It\'s not an "exact band" but a \n        maximum requirement\n    - The Comet constraint is not modeled by any of the rules. A "restricted positions" type rules should be made \n        to accommodate this\n    - This will allow the generate_rule/eliminate_sectors methods to take in a set of constraints and compare it to\n        themselves to determine if generating a certain rule makes sense\n    - It might make sense to unify SelfRule and RelationRule in the process\n        - The three types of self rules are equivalent to the three types of relation rules\n        - However, the comet-type rule would not be equivalent to any relation

In [2]:
import itertools
import random
import json
from enum import Enum
from abc import *

In [3]:
from server.utilities import permutations_multi, fill_no_touch

In [4]:
from server.board import *
from server.board_type import *

In [5]:
from server.game import RuleQualifier

In [6]:
class Rule(ABC):
    @abstractmethod
    def is_satisfied(self, board):
        """
        Checks whether a board meets this constraint. Returns true if constraint is met.
        """
        pass
    
    @abstractmethod
    def is_immediately_limiting(self):
        """
        Returns true if this constraint limits the positions the space object can be in
        even when no objects are on the board.
        """
        pass

    @abstractmethod
    def disallowed_sectors(self):
        """
        If the constraint is immediately limited, returns the list of sector that the 
        space object relevant to the constraint is not allowed to be in. Otherwise,
        returns [].
        """
        return []
    
    @abstractmethod
    def fill_board(self, board, num_objects):
        """
        Given a board and the number of each space object that must appear on it,
        fills the board in all ways possible given this constraint.
        
        board: A partially filled Board
        num_objects: A dictionary mapping space objects to the number of times they
            are supposed to appear in the board.
            
        Returns: a list of boards that contain the same objects as the original board,
            plus filling in objects to meet this constraint. Returns all possible
            such boards.
        """
        pass
    
    @abstractmethod
    def affects(self):
        """
        Returns a list of space objects that are affected by this rule. Note that this is
        not the same as all of the space objects named in the constraint - e.g.
            "All gas clouds are next to an empty sector" does not affect empty sectors,
            since the other empty sectors may be wherever they like
        """
        pass
    
    @abstractmethod
    def completes(self):
        """
        Returns a list of all space object that this constraint can "complete" - that is, 
        when generating all possible boards using fill_board, this constraint will add in 
        every object of that type.
        """
        pass
    
    @abstractmethod
    def adds(self):
        """
        Returns a list of all space objects that can be added during fill_board
        """
        pass
    
    @abstractmethod
    def space_objects(self):
        """
        Returns all the space objects involved in this rule
        """
        pass
    
    @classmethod
    @abstractmethod
    def generate_rule(cls, board, *space_objects):
        """
        Generates a rule of this type for a particular board and space objects to relate 
        to each other. Returns None if no such rule exists.
        """
        pass
    
    @abstractmethod
    def code(self):
        """
        Return compressed string code that represents this rule
        """
        pass
    
    @classmethod
    @abstractmethod
    def parse(cls, s):
        """
        Parse a compressed string representation s into a rule of this type
        """
        pass
    
    @abstractmethod
    def text(self, board):
        """
        Returns a readable text version of this rule given an input Board board.
        """
        pass
    
    @abstractmethod
    def to_json(self, board):
        """
        Create a json representation of this rule, given a particular board. The json
        representation should have the following fields:
            - ruleType: type of the rule
            - If the rule is a self-rule:
                - spaceObject: the space object in the rule
            - If the rule is a relation-rule:
                - spaceObject1: the space object in the rule
                - spaceObject2: the space object which spaceObject1 is related to in the rule
            - categoryName: the name of the category the rule is in (based on types of objects)
            - text: a readable text representation of the rule
            - other rule-specific fields   
        """
        pass

In [7]:
class RelationRule(Rule):
    def space_object1(self):
        return self.space_object1
    
    def space_object2(self):
        return self.space_object2
    
    def space_objects(self):
        return [self.space_object1(), self.space_object2()]
    
    @classmethod
    @abstractmethod
    def eliminate_sectors(cls, board, data, space_object1, space_object2):
        """
        Create a rule that will eliminate possible positions of space_object1, where 
        space_object1 and data.elimination_object are ambiguous on survey/target
        
        Only the sectors in data.need_eliminated are ambiguous, and the sectors in 
        data.already_eliminated have been eliminated by other rules. To be viable,
        this rule must eliminate at least data.minimum sectors, and if possible 
        should eliminate data.goal sectors.
        """
        pass

In [8]:
class SelfRule(Rule):
    def space_object(self):
        return self.space_object
    
    def space_objects(self):
        return [self.space_object()]

In [119]:
class AdjacentRule(RelationRule):
    """
    A rule stating that two objects are or aren't adjacent to one another
    """
    def __init__(self, space_object1, space_object2, qualifier):
        self.space_object1 = space_object1
        self.space_object2 = space_object2
        self.qualifier = qualifier
        
    def __repr__(self):
        return "<" + self.qualifier.name + " " + repr(self.space_object1) + " adjacent to " \
                + repr(self.space_object2) + ">"
    
    def __str__(self):
        return str(self.qualifier) + " " + self.space_object1.name() + " is adjacent to " + \
                self.space_object2.one() + " " + self.space_object2.name() + "."
    
    def text(self, board):
        num_object1 = board.num_objects()[self.space_object1]
        num_object2 = board.num_objects()[self.space_object2]
        return self.qualifier.for_object(self.space_object1, num_object1) + " adjacent to " + \
                self.space_object2.any_of(num_object2) + "."
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            if self.qualifier == other.qualifier:
                if self.qualifier is RuleQualifier.NONE or self.qualifier is RuleQualifier.AT_LEAST_ONE:
                    # Mutual rules - i.e. space objects can be swapped
                    return self.space_object1 == other.space_object1 and self.space_object2 == other.space_object2 or \
                            self.space_object1 == other.space_object2 and self.space_object2 == other.space_object1
                else:
                    # One way rules - i.e. space objects cannot be swapped
                    return self.space_object1 == other.space_object1 and self.space_object2 == other.space_object2
            else:
                return False
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def is_satisfied(self, board):
        adjacent_idxs = [i for i in range(len(board)) if board[i] is self.space_object1
                            and board[i-1] is self.space_object2 or board[i+1] is self.space_object2]
        
        if self.qualifier is RuleQualifier.NONE:
            return len(adjacent_idxs) == 0
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return len(adjacent_idxs) > 0
        else:
            return len(adjacent_idxs) == board.num_objects()[self.space_object1]
    
    def is_immediately_limiting(self):
        return False

    def disallowed_sectors(self):
        return []
    
    def _fill_board_none(self, board, num_objects):
        if not self.is_satisfied(board):
            # There are already two adjacent in this board, cannot meet rule
            return []
        
        num_obj1 = num_objects[self.space_object1]
        num_obj2 = num_objects[self.space_object2]
        
        for obj in board:
            if obj is self.space_object1:
                num_obj1 -= 1
            elif obj is self.space_object2:
                num_obj2 -= 1
        
        board_perms = fill_no_touch({self.space_object1: num_obj1, self.space_object2: num_obj2}, board)
        return [Board(board_objects) for board_objects in board_perms]
    
    
    def _fill_board_every(self, board, num_objects, num_objects_left, start_i=0):
        # num_objects: how many should be on the board starting from start_i
        # num_objects_left: how many still need to be placed
        num_obj1 = num_objects[self.space_object1]
        num_obj2 = num_objects[self.space_object2]
        
        num_obj1_left = num_objects_left[self.space_object1]
        num_obj2_left = num_objects_left[self.space_object2]
                
        new_num_objects = num_objects.copy()
        new_num_objects_left = num_objects_left.copy()

        if num_obj1 == 0:
            return [ board ]
        
        new_boards = []
        for i in range(start_i, len(board)):
            obj = board[i]
            is_obj1 = obj is self.space_object1
            # Attempt to fill each position with an object1 if possible
            if obj is None or is_obj1:
                options = 0
                if is_obj1 or num_obj1_left > 0:
                    if board[i-1] is self.space_object2 or board[i+1] is self.space_object2:
                        options += 1
                        # If there is already an obj2 next to it, fill with obj1 and proceed
                        board_copy = board.copy()
                        board_copy[i] = self.space_object1
                        new_num_objects[self.space_object1] = num_obj1 - 1
                        new_num_objects[self.space_object2] = num_obj2
                        new_num_objects_left[self.space_object1] = num_obj1_left - (not is_obj1)
                        new_num_objects_left[self.space_object2] = num_obj2
                        new_boards.extend(self._fill_board_every(board_copy, new_num_objects, new_num_objects_left, i+1))
                    elif num_obj2_left > 0:
                        # Otherwise there must be obj2 left to use
                        if board[i-1] is None and board[i-2] is not self.space_object1:
                            # Do not put an obj2 on the left if there is a obj1
                            # already to the left of that, to avoid duplicate boards
                            options += 1
                            board_copy = board.copy()
                            board_copy[i] = self.space_object1
                            board_copy[i-1] = self.space_object2
                            new_num_objects[self.space_object1] = num_obj1 - 1
                            new_num_objects[self.space_object2] = num_obj2 - 1
                            new_num_objects_left[self.space_object1] = num_obj1_left - (not is_obj1)
                            new_num_objects_left[self.space_object2] = num_obj2_left - 1
                            new_boards.extend(self._fill_board_every(board_copy, new_num_objects, new_num_objects_left, i+1))
                    
                        if board[i+1] is None and (i+2 < len(board) or board[i+2] is not self.space_object1):
                            # Do not put an obj2 on the right if there is a obj1
                            # to the right of that, to avoid duplicate boards
                            # Only follow this restriction if we are looking at a space object that we placed 
                            # under this algorithm, not one that existed before
                            options += 1
                            board_copy = board.copy()
                            board_copy[i] = self.space_object1
                            board_copy[i+1] = self.space_object2
                            new_num_objects[self.space_object1] = num_obj1 - 1
                            new_num_objects[self.space_object2] = num_obj2 - 1
                            new_num_objects_left[self.space_object1] = num_obj1_left - (not is_obj1)
                            new_num_objects_left[self.space_object2] = num_obj2_left - 1
                            new_boards.extend(self._fill_board_every(board_copy, new_num_objects, new_num_objects_left, i+2))

                if is_obj1 and options == 0:
                    # Unable to place an obj2 next to an existing obj1
                    return []
            
        return new_boards
    
    def fill_board(self, board, num_objects):
        if self.qualifier is RuleQualifier.NONE:
            return self._fill_board_none(board, num_objects)
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            # Not yet supported
            return None
        else:
            num_objects_left = num_objects.copy()
            for obj in board:
                if obj is self.space_object1 or obj is self.space_object2:
                    num_objects_left[obj] -= 1
                
            return self._fill_board_every(board, num_objects, num_objects_left)
            
    def affects(self):
        if self.qualifier is RuleQualifier.NONE:
            return [ self.space_object1, self.space_object2 ]
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return []
        else:
            return [ self.space_object1 ]
    
    def completes(self):
        pass
    
    def adds(self):
        pass
    
    @classmethod
    def generate_rule(cls, board, space_object1, space_object2):
        # Some are already constrained, don't generate these rules
        if (space_object1, space_object2) in {
            (SpaceObject.GasCloud, SpaceObject.Empty),
            (SpaceObject.PlanetX, SpaceObject.DwarfPlanet),
            (SpaceObject.DwarfPlanet, SpaceObject.PlanetX),
            (SpaceObject.BlackHole, SpaceObject.PlanetX),
            (SpaceObject.PlanetX, SpaceObject.BlackHole),
            (SpaceObject.BlackHole, SpaceObject.Empty)
        }:
            return None

        num_adjacent = 0
        
        # Count how many object1s are adjacent to object2s 
        for i, obj in enumerate(board):
            if obj is space_object1:
                if board[i-1] is space_object2 or board[i+1] is space_object2:
                    num_adjacent += 1
        
        num_object1 = board.num_objects()[space_object1]
        num_object2 = board.num_objects()[space_object2]
        
        # Create rule options
        if num_adjacent == 0:
            # None were adjacent
            qualifier_options = [RuleQualifier.NONE]
        elif num_adjacent < num_object1:
            # Some were adjacent
            qualifier_options = [RuleQualifier.AT_LEAST_ONE]
        else:
            # Every one was adjacent - also means at least one was 
            qualifier_options = [RuleQualifier.AT_LEAST_ONE, RuleQualifier.EVERY]
        
        # At least one just means every
        if num_object1 == 1:
            qualifier_options = [option for option in qualifier_options \
                                if option is not RuleQualifier.AT_LEAST_ONE]
        
        # Finding one object2 finds all object1s
        if num_object1 >= 2 * num_object2:
            qualifier_options = [option for option in qualifier_options \
                                if option is not RuleQualifier.EVERY]
            
        if len(qualifier_options) == 0:
            return None
        
        qualifier = random.choice(qualifier_options)
        return AdjacentRule(space_object1, space_object2, qualifier)
    
    @classmethod
    def eliminate_sectors(cls, space_object1, space_object2, data, board):  
        """
        Create a rule that will eliminate possible positions of space_object1, where 
        space_object1 and data.elimination_object are ambiguous on survey/target
        
        Only the sectors in data.need_eliminated are ambiguous, and the sectors in 
        data.already_eliminated have been eliminated by other rules. To be viable,
        this rule must eliminate at least data.minimum sectors, and if possible 
        should eliminate data.goal sectors.
        """
        obj1_num_adjacent = 0
        el_adjacent = set()
        
        num_obj1 = board.num_objects()[space_object1]
        num_obj2 = board.num_objects()[space_object2]
        num_el = len(data.need_eliminated)
        
        # If the eliminated object is space object2, then space object1 "looks like" space object 2
        adjacent_objs = [space_object2]
        if data.elimination_object is space_object2:
            adjacent_objs.append(space_object1)
        
        # Count how many are adjacent
        for i, obj in enumerate(board):
            # An "object 2" is adjacent to this sector
            if board[i-1] in adjacent_objs or board[i+1] in adjacent_objs:
                if obj is space_object1:
                    # Count an object 1 being adjacent
                    obj1_num_adjacent += 1
                elif obj is data.elimination_object and i in data.need_eliminated:
                    # Add to list of sectors where the eliminated object is adjacent
                    el_adjacent.add(i)
                    
        el_num_adjacent = len(el_adjacent)

        # Uncomment to allow "Planet X is adjacent to a <obj>" type rules
#         if obj1_num_adjacent == num_obj1 and el_num_adjacent < num_el:
#             el_positions = set(i for i, obj in enumerate(board) if obj is eliminated_object)
#             eliminated = el_positions - el_adjacent - previously_eliminated
#             if len(eliminated) >= minimum:
#                 rule = AdjacentRule(space_object1, space_object2, RuleQualifier.EVERY, num_obj1, num_obj2)
#                 return eliminated, rule, eliminated, rule
        
        # Only using "not adjacent" rules to avoid too powerful rules
        # There must be no object 1's ajacent to an 'object 2' 
        if obj1_num_adjacent == 0 and el_num_adjacent > 0:
            eliminated = el_adjacent - data.already_eliminated
            # Must eliminate data.minimum new sectors
            if len(eliminated) >= data.minimum:
                rule = AdjacentRule(space_object1, space_object2, RuleQualifier.NONE)
                return eliminated, rule
        
        return None, None
    
    def code(self):
        return "A" + str(self.space_object1) + str(self.space_object2) + self.qualifier.code()
    
    @classmethod
    def parse(cls, s):
        space_object1 = SpaceObject.parse(s[1])
        space_object2 = SpaceObject.parse(s[2])
        qualifier = RuleQualifier.parse(s[3])
        return cls(space_object1, space_object2, qualifier)
    
    def to_json(self, board):
        return {
            "ruleType": "ADJACENT",
            "spaceObject1": self.space_object1.to_json(),
            "spaceObject2": self.space_object2.to_json(),
            "qualifier": self.qualifier.to_json(),
            "categoryName": self.category_name(),
            "text": self.text(board)
        }

In [130]:
b = Board([None, None, SpaceObject.GasCloud, SpaceObject.Empty, None])

In [131]:
g = AdjacentRule(SpaceObject.GasCloud, SpaceObject.Empty, RuleQualifier.EVERY)

In [132]:
g.fill_board(b, sector_types[12].num_objects)

[<Board G-GEE>, <Board GEGE->, <Board EGGE->, <Board --GEG>]

In [180]:
p = AdjacentRule(SpaceObject.PlanetX, SpaceObject.BlackHole, RuleQualifier.NONE)